Will build some of the initial code here, thinking about how we can be flexible to what sort of strategies we want to use. Want to consider that when we move to do things like "liking" or "following" people, want to make sure that this is done flexibly so that we can turn it on or off.

In [ ]:
from InstagramAPI import InstagramAPI
from pathlib import Path
import json

In [ ]:
class instabot():
    """
    The class we want for our instagrambot
    
    I think the way I will build this will be to have a lot of generic methods that
    abstract the API and that can then intertwine with each other in a simple way,
    avoiding a lot of extra code and stuff
    
    This will also let us develop pretty flexible strategies that just
    call in this class and then run through the results, not making any
    API calls themselves, and then decide what to do with certain users
    
    It will be important to keep lists of user IDs so that we don't bother
    the same people time and again as will likely be wasted effort
    """
    
    def __init__(self, credentials_path):
        # First we want to set up the credentials that we will use to connect
        self.credentials_path = credentials_path
        self.credentials = self.load_credentials()
        
        # Then load the API
        self.API = self.init_api()
        
        
    def load_credentials(self):
        """
        Small thing to load the credentials json
        """
        with self.credentials_path.open() as cred_file:
            return json.load(cred_file)
        
    def init_api(self):
        """
        Initialise the API and try to log in
        """
        API = InstagramAPI(self.credentials['username'], self.credentials['password'])
        have_loggedin = API.login()
        
        if have_logged_in:
            return API
        else:
            # Not sure what other sort of error to raise as we just return True / False
            raise ValueError('Failed to login - credentials probably incorrect')
    
    def get_last(self, function, *args):
        """
        Just a shortcut to get the last Json to clean stuff up
        Maybe the *args thing is a bit dangerous but I think this lazy
        method will work pretty OK, and avoid some mess in the code
        """
        _ = getattr(self.API, function)(*args)
        return self.API.LastJson
    
    def get_user_info(self, user_id):
        """
        Wrapper to get the key stats of some user ID, especially the follower count
        """
        results = self.get_last('getUsernameInfo', user_id)['user']
        
        desired_info = ['follower_count', 'following_count', 'media_count']
        return {desired_key : results[key] for key in desired_info}
    
    def get_hashtag_content(self, hashtag):
        """
        Wrapper to retrieve some stories from a hashtag
        I guess this should just return as is, and then later 
        we include logic to decide which post to go for
        Note that we just stick with the default number of posts as we probably 
        don't want to overload
        """
        return self.get_last('getHashtagFeed', hashtag)['ranked_items']
    
    def get_story_likes(self, media_id):
        """
        Wrapper to use a media_id to pull the likes
        """
        return self.get_last('getMediaLikers', media_id)['user']
    
    def get_story_comments(self, media_id):
        """
        Wrapper to use a media_id to pull the comments
        """
        return self.get_last('getMediaComments', media_id)['comments']
    
    def get_user_history(self, user_id):
        """
        Wrapper to get the user feed so that we can check things like
        the last post they have made
        """
        return self.get_last('getUserFeed', user_id)['user']
    
    def init_exlclusion_list(self, save_loc = None):
        """
        A wrapper where we will initialise some exclusion list
        of users that we want to ignore, where we can provide a path
        to a list that we want to keep - note that we should provide a json
        """
        if save_loc:
            with save_loc.open() as list_file:
                self.exclusion_list = json.load(list_file)['user_ids']
                
        else:
            self.exclusion_list = []
        
    def update_exclusion_list(self, user_id):
        """
        Just appending /extending
        """
        if type(user_id) == list:
            self.exclusion_list.extend(user_id)
        else:
            self.exclusion_list.append(user_id)
    
    def save_exclusion_list(self, save_loc):
        with save_loc.open() as list_file:
            json.dump({'user_ids' : self.exclusion_list})